In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from selenium.common.exceptions import NoSuchElementException
import collections

In [112]:
chromedriver = '/Users/alex/Downloads/chromedriver 4'
browser = webdriver.Chrome(chromedriver)

In [98]:
url = 'https://leesa.com'
browser.get(url + '/mattresses')


In [67]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [35]:
brand, model, rating, review, warranty, trial, reviewer_name,\
review_location, review_date = get_reviews('tempur pedic')

0/481 reviews added
100/481 reviews added
200/481 reviews added
300/481 reviews added
400/481 reviews added
0/1150 reviews added
100/1150 reviews added
200/1150 reviews added
300/1150 reviews added
400/1150 reviews added
500/1150 reviews added
600/1150 reviews added
700/1150 reviews added
800/1150 reviews added
900/1150 reviews added
1000/1150 reviews added
1100/1150 reviews added
0/2505 reviews added
100/2505 reviews added
200/2505 reviews added
300/2505 reviews added
400/2505 reviews added
500/2505 reviews added
600/2505 reviews added
700/2505 reviews added
800/2505 reviews added
900/2505 reviews added
1000/2505 reviews added
1100/2505 reviews added
1200/2505 reviews added
1300/2505 reviews added
1400/2505 reviews added
1500/2505 reviews added
1600/2505 reviews added
1700/2505 reviews added
1800/2505 reviews added
1900/2505 reviews added
2000/2505 reviews added
2100/2505 reviews added
2200/2505 reviews added
2300/2505 reviews added
2400/2505 reviews added
2500/2505 reviews added
0/52

In [42]:
df = pd.DataFrame()

df['brand'] = ['Tempur Pedic'] * len(df)
df['model'] = model
df['rating'] = rating
df['review'] = review
df['warranty'] = warranty
df['trial'] = trial
df['reviewer_name'] = reviewer_name
df['review_location'] = review_location
df['review_date'] = review_date

df.to_csv('tempur_pedic_all_reviews.csv')

In [48]:
df

,brand,model,rating,review,warranty,trial,reviewer_name,review_location,review_date
0,Tempur Pedic,TEMPUR-Cloud,5.0,"Love love Tempur pedic, I always use this prod...",10-Year,90,Danny Wardana,Birmingham,"Dec 16, 2020"
1,Tempur Pedic,TEMPUR-Cloud,5.0,I recently began experiencing general body ach...,10-Year,90,Professional Sleeper,Atlanta,"Dec 6, 2020"
2,Tempur Pedic,TEMPUR-Cloud,5.0,It's a pleasure sleeping on it. I only wish I ...,10-Year,90,Best sleep,L.I,"Nov 28, 2020"
3,Tempur Pedic,TEMPUR-Cloud,5.0,We love our new mattress! It's night and day c...,10-Year,90,Amy,Hagerstown,"Nov 22, 2020"
4,Tempur Pedic,TEMPUR-Cloud,5.0,Did not have to go shopping around...knew from...,10-Year,90,Rested!,St. Clair Shores,"Nov 12, 2020"
...,...,...,...,...,...,...,...,...,...
5595,Tempur Pedic,TEMPUR-breeze,1.0,I wake up every morning in more pain then the ...,10-Year,90,Bad sleep,Pampa,"Jan 12, 2020"
5596,Tempur Pedic,TEMPUR-breeze,1.0,Wife and I decided to upgrade from a Queen to ...,10-Year,90,NO SLEEP Jason,Lafayette,"Nov 28, 2019"
5597,Tempur Pedic,TEMPUR-breeze,1.0,I bought this mattress because It was suppose ...,10-Year,90,Do not buy !!! This mattress is hot !!!,New Orleans,"Oct 5, 2019"
5598,Tempur Pedic,TEMPUR-breeze,1.0,Might work if you sleep on your back but if yo...,10-Year,90,Eli,New York,"Jun 28, 2019"


In [36]:
def get_reviews(company, number_of_reviews=0):
    brand = []
    model = []
    rating = []
    review = []
    warranty = []
    trial = []
    reviewer_name = []
    review_location = []
    review_date = []
    
    #INSERT INTO Mattress_Info (brand, model, size, style, price)
    #VALUES (brand, model, size, style, price)
    
    #INSERT INTO Reviews (brand, model, review, additional_info, name, location, review_date)
    #VALUES (brand, model, review, additional_info)
    
    
    if company.lower() == 'tempur pedic':
        # tempur pedic url
        url = 'https://www.tempurpedic.com/shop-mattresses/'
        browser.get(url)
        
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        mattresses = soup.find_all(attrs={'aria-label': 'Mattress Model'})
        
        for i in range(len(mattresses)):
            # Finding the link of the specific mattress on the page and redirecting to that page
            browser.get(mattresses[i].find('a').get('href'))
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            
            # getting all reviews
            total_reviews = soup.find(attrs={'class': 
                                     'star-rating__review-number configurator__star-rating__review-number'}).text
            total_reviews = int(re.sub(' Reviews', '', total_reviews))


            time.sleep(5)
            
            #browser.find_element_by_xpath('//*[@id="reviews__body"]/div[2]/div[1]/div[2]').click()
            # Clicking the review sort by drop down menu below
            element = browser.find_element_by_xpath('//*[@id="review-sort"]')
            browser.execute_script("arguments[0].click();", element)
            #browser.find_element_by_xpath('').click()
            # Clicking on 'Newest' from the review sort by dropdown menu to sort by Newest Reviews
            element = browser.find_element_by_xpath('//*[@id="review-sort"]/option[3]')
            browser.execute_script("arguments[0].click();", element)
            #browser.find_element_by_xpath('//*[@id="review-sort"]/option[3]').click()

            viewable_reviews = len(soup.find_all(attrs={'aria-label': 'Product Review'}))
            count = 0
            while count < total_reviews:
                browser.maximize_window()
                if viewable_reviews != total_reviews:
                    try:
                        browser.find_element_by_xpath('//*[@id="reviews__body"]').click()
                        page_data = browser.page_source
                        soup = BeautifulSoup(page_data, 'html.parser')
                        element = browser.find_element_by_xpath('//*[@id="reviews__body"]/div[2]/div[2]/div[2]')
                        browser.execute_script("arguments[0].click();", element)
                        viewable_reviews = len(soup.find_all(attrs={'aria-label': 'Product Review'}))
                        count=viewable_reviews
                    except NoSuchElementException:
                        break

            page_data = browser.page_source
            soup = BeautifulSoup(page_data, 'html.parser')
            for x in range(total_reviews):
                brand.append('Tempur Pedic')
                model.append(soup.find(class_='configurator__title').text[:-1])
                warranty.append('10-Year')
                trial.append(90)
                rating.append(float(soup.find_all('div', attrs={'class':'star-rating__number product-reviews-filter__star-rating__number'})[x].text))
                review.append(re.sub('[\n\t]', '', soup.find_all('div', attrs={'class': 'customer-review__review-item__copy'})[x].text))
                reviewer_name.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'})[x].text.split(',')[0])
                review_location.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'})[x].text.split(',')[1])
                review_date.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__date'})[x].text)

                if x % 100 == 0:
                    print(str(x) + '/' + str(total_reviews) + ' reviews added')
                
                
    if company.lower() == 'casper':
        url = 'https://casper.com'
        browser.get(url + '/mattresses')
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        # Gathering the spot on the page which contains links for all individual matresses
        mattresses = soup.find_all('div', attrs={'class': 'sc-1fi10si-50 gzIyoR'})
        
        for mattress in range(len(mattresses)):
            # Going to each individual page for each mattress
            browser.get(url + mattresses[mattress].find('a').get('href'))
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            
            # Getting the total review counts for each mattress on the mattress page
            total_reviews = int(re.sub('\W', '', soup.find_all(attrs={'data-qa': 'render-total-count'})[0].text))
            count = 0
            # Changing the total review count to 2000 if the amount of reviews is greater than 2000
            if total_reviews > number_of_reviews:
                total_reviews = number_of_reviews

            trial_ = int(soup.find_all(class_='NavItemText-sc-15rw0ng-6 dMJZWe')[1].text[:3])
            warranty_ = soup.find_all(class_='NavItemText-sc-15rw0ng-6 dMJZWe')[2].text[:7]
            model_type = soup.find_all(attrs={'class': 'x900hc-3 dbmrVC'})[1].text + ' Mattress'

            # Clicking to get to the page with full reviews
            browser.find_element_by_link_text('See all reviews').click()
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            
            # Changing the sorting method of reviews to most recent
                # Clicking the sort by drop down menu
            element = browser.find_element_by_xpath('//*[@id="reviews"]/div[1]/div[2]/div/div/button/span[1]')
            browser.execute_script("arguments[0].click();", element)
                # Clicking the Most Recent option in the sort by drop down menu
            element = browser.find_element_by_xpath('//*[@id="reviews"]/div[1]/div[2]/div/div/ul/li[2]/span')
            browser.execute_script("arguments[0].click();", element)

            while count < total_reviews:
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                count+=len(soup.find_all('div', attrs={'class':'rn6em9-0 innTNk'}))
                for i in range(len(soup.find_all('div', attrs={'class':'rn6em9-0 innTNk'}))):
                    brand.append('Casper')
                    model.append(model_type)
                    trial.append(trial_)
                    warranty.append(warranty_)
                    review.append(soup.find_all('div', attrs={'class':'rn6em9-0 innTNk'})[i].text)
                    rating.append(int(soup.find_all(class_='lnmqqa-3 ncHUI')[i].text[0]))
                    reviewer_name.append(re.match('\w+', soup.find_all(class_='sc-55wbfa-1 hoZTLI')[i].text).group().replace('Checkmark', ''))
                    if len(soup.find_all(class_='sc-55wbfa-0 bUotLN')) != 10:
                        review_location.append(soup.find_all(class_='sc-55wbfa-2 dVruIH')[i].text.split('Owner')[1])
                    elif len(soup.find_all(class_='sc-55wbfa-0 bUotLN')) == 10:   
                        review_location.append(soup.find_all(class_='sc-55wbfa-0 bUotLN')[i].text)
                    review_date.append(soup.find_all(class_='imungi-8 fmEyIG')[0].text.replace('Review left on ', ''))
                if count % 100 == 0:
                    print(str(count) + '/' + str(total_reviews) + ' reviews added')

                element = browser.find_element_by_xpath('//*[@id="reviews"]/div[2]/div[2]/a[3]')
                browser.execute_script("arguments[0].click();", element)
                
    if company.lower() == 'leesa':
        url = 'https://www.leesa.com'
        browser.get(url + '/pages/compare-mattresses')
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        mattress_1 = '//*[@id="shopify-section-compare-page-header"]/div[4]/div/div[1]/a'
        mattress_2 = '//*[@id="shopify-section-compare-page-header"]/div[4]/div/div[2]/a'
        mattress_3 = '//*[@id="shopify-section-compare-page-header"]/div[4]/div/div[3]/a'

        for link in [mattress_1, mattress_2, mattress_3]:
            browser.find_element_by_xpath(link).click()
            soup = BeautifulSoup(browser.page_source, 'html.parser')
        
            # Clicking to go to reviews
            element = browser.find_element_by_xpath('//*[@id="edge-fix"]/div/div/div[1]/div/div')
            browser.execute_script("arguments[0].click();", element)

            time.sleep(5)
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            try:
                browser.find_element_by_xpath('//*[@id="privy-inner-container"]/div[38]/div/div/div[2]/img').click()
            except:
                pass
            time.sleep(5)
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            
            try:
                # Clicking drop down menu on reviews page to switch to recent
                element = browser.find_element_by_xpath('//*[@id="bv-dropdown-title-reviews"]')
                browser.execute_script("arguments[0].click();", element)
                # Click most recent on drop down menu
                time.sleep(5)
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                browser.find_element_by_xpath('//*[@id="data-bv-dropdown-item-mostRecent"]').click()
            except:
                pass
            # Clicking body of page if needed because of pop-up
            #browser.find_element_by_xpath('//*[@id="privy-inner-container"]/div[26]/div/div/div[2]/img').click()
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            total_reviews = int(soup.find(class_='bv-rating-label bv-text-link bv-focusable').text.split()[0])
            
            if total_reviews > number_of_reviews:
                total_reviews = number_of_reviews
            count = 0

            while count < total_reviews:
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                count+= len(soup.find_all(class_='bv-rating bv-rating-stars bv-rating-stars-off'))

                for i in range(len(soup.find_all(class_='bv-rating bv-rating-stars bv-rating-stars-off'))):
                    brand.append('leesa')
                    model.append(soup.find_all(attrs={'class': 'text__center color__green text__weight__semibold'})[0].text + ' Mattress')
                    review.append(soup.find_all('div', attrs={'class': 'bv-content-summary-body-text'})[i].text)
                    rating.append(int(soup.find_all(class_='bv-rating bv-rating-stars bv-rating-stars-off')[i]['title'][0]))
                    reviewer_name.append(soup.find_all(attrs={'itemprop': 'author'})[i].text)
                    warranty.append('10-Year')
                    trial.append(100)
                    review_location.append(soup.find_all(attrs={'class':'bv-author-location'})[i].text)
                    review_date_current = soup.find_all('span', attrs={'class': 'bv-content-datetime-stamp'})[i].text[:-2]
                    today = datetime.today()
                    try:
                        review_dt = int(review_date_current.split()[0])
                    except:
                        review_dt = 1
                    if 'days' in review_date_current:
                        review_dt = today - timedelta(days=review_dt)
                        review_dt = review_dt.strftime('%m-%d-%y')
                    elif 'month' in review_date_current:
                        review_dt = today - relativedelta(months=review_dt)
                        review_dt = review_dt.strftime('%m-%d-%y')
                    elif 'year' in review_date_current:
                        review_dt = today - relativedelta(years=review_dt)
                        review_dt = review_dt.strftime('%m-%d-%y')

                    review_date.append(review_dt)

                browser.find_element_by_xpath('//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a/span[2]').click()
                if count % 100 == 0:
                    print('{0}/{1} reviews added'.format(count, total_reviews))
            browser.get(url + '/pages/compare-mattresses')
                
    if company.lower() == 'purple':
        url = 'https://purple.com'
        browser.get(url + '/mattresses/')
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        time.sleep(5)
        
        for i in range(3):
            mattress = soup.find_all('div', attrs={'class': 'product-info-container ta-left m-b-4'})[i].find('a').get('href')
            browser.get(url+mattress)
            soup = BeautifulSoup(browser.page_source, 'html.parser')

            # Click dropdown menu
            element = browser.find_element_by_xpath('//*[@id="stamped-sort-select"]')
            browser.execute_script("arguments[0].click();", element)

            # Click most recent on drop down menu
            element = browser.find_element_by_xpath('//*[@id="stamped-sort-select"]/option[3]')
            browser.execute_script("arguments[0].click();", element)
            time.sleep(5)

            total_reviews = int(soup.find('li', attrs={'id': 'tab-reviews'}).get('data-count'))
            if total_reviews > number_of_reviews:
                total_reviews = number_of_reviews
            count = 0

            while count < total_reviews:
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                count+= len(soup.find_all(class_='stamped-starratings stamped-review-header-starratings'))
                for i in range(len(soup.find_all(class_='stamped-starratings stamped-review-header-starratings'))):
                    brand.append('purple')
                    model.append(soup.find('h1', attrs={'class': 'fs-h-3'}).text)
                    c = collections.Counter(soup.find_all(class_='stamped-starratings stamped-review-header-starratings')[i])
                    c_2 = list(c)
                    c_key = c_2[1]
                    rating.append(collections.Counter(soup.find_all(class_='stamped-starratings stamped-review-header-starratings')[i])[c_key])
                    review.append(soup.find_all(class_='stamped-review-content-body')[i].text)
                    warranty.append('10-Year')
                    trial.append(100)
                    reviewer_name.append(soup.find_all('div', attrs={'class': 'author'})[i])
                    review_location.append(soup.find_all('div', attrs={'class': 'review-location'})[i].text)
                    review_date.append(soup.find_all('div', attrs={'class': 'created'})[i].text)

                element = browser.find_element_by_xpath('//*[@id="stamped-main-widget"]/div/div[3]/div[5]/ul/li[14]/a')
                browser.execute_script("arguments[0].click();", element)

            browser.get(url + '/mattresses/')
            soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    if company.lower() == 'serta':
        url = 'https://www.serta.com'
        browser.get(url + '/mattresses')
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        mattress_links = soup.find_all('div', attrs={'class': 'ProductGrid'})[0].find_all('a')
        
        for i in range(len(mattress_links)):
            browser.get(url + mattress_links[i].get('href'))
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            time.sleep(4)
            try:
                browser.find_element_by_xpath('//*[@id="hero"]/div[2]/div[3]/div/div/form/div/div/div/button').click()
                browser.find_element_by_xpath('//*[@id="hero"]/div[2]/div[3]/div/div/form/div/div/div/ul/li[4]/button').click()
                total_reviews = int(soup.find_all(class_='bv_numReviews_component_container')[0].text.split('(')[1].replace(')', ''))
                if total_reviews > number_of_reviews:
                    total_reviews = number_of_reviews
                count = 0

                while count < total_reviews:
                    soup = BeautifulSoup(browser.page_source, 'html.parser')
                    count += len(soup.find_all(class_='BVRRReviewTextParagraph BVRRReviewTextFirstParagraph BVRRReviewTextLastParagraph'))

                    for i in range(len(soup.find_all(class_='BVRRReviewTextParagraph BVRRReviewTextFirstParagraph BVRRReviewTextLastParagraph'))):
                        brand.append('serta')
                        model.append(soup.find('h1', attrs={'class': 'pdp__product-heading'}).text)
                        rating.append(int(soup.find_all('img', attrs={'src': 'https://serta.ugc.bazaarvoice.com/0245-en_us/5_0/9/rating.gif'})[i]['alt'][0]))
                        review.append(soup.find_all(class_='BVRRReviewTextParagraph BVRRReviewTextFirstParagraph BVRRReviewTextLastParagraph')[i].text)
                        warranty.append('10-Year')
                        trial.append(120)
                        reviewer_name.append(soup.find_all(class_='BVRRNickname')[i].text)
                        if 'BVRRUserLocationContainer' in soup.find_all(class_='BVRRContextDataContainer')[i]['class']:
                            review_location.append(re.search(':\w+', 
                                             soup.find_all(class_='BVRRContextDataContainer')[i].text)\
                                   .group().split('A')[0].split(':')[1])
                        else:
                            review_location.append('')
                        review_date.append(soup.find_all(class_='BVRRValue BVRRReviewDate')[i].text)

                    browser.find_element_by_xpath('//*[@id="BVRRDisplayContentFooterID"]/div/span[6]').click()
                    if count % 10 == 0:
                        print(str(count) + '/' + str(total_reviews) + ' reviews added')
            except:
                continue
                
    if company.lower() == 'sealy':
        url = 'https://www.sealy.com/online-mattresses/cocoon-sealy-chill/v/45/'
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        # Clicking to get to reviews
        browser.find_element_by_xpath('//*[@id="product-detail"]/div[1]/div/div/div/div[2]/button[2]').click()
        # Clicking dropdown to sort by new
        browser.find_element_by_xpath('//*[@id="review-sort"]').click()
        # Clicking new in the dropdown menue to sort by new
        browser.find_element_by_xpath('//*[@id="review-sort"]/option[3]').click()

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        total_reviews = int(soup.find('div', attrs={'class': 'customer-review__number'}).text.split('+')[0])
        if total_reviews>2000:
            total_reviews = 2000
        count = 0

        visible_reviews = len(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'}))

        while count < total_reviews:
            if visible_reviews < total_reviews:
                element = browser.find_element_by_xpath('//*[@id="reviews__body"]/div[2]/div[2]/div[2]')
                browser.execute_script("arguments[0].click();", element)
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                visible_reviews = len(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'}))

            soup = BeautifulSoup(browser.page_source, 'html.parser')     
            for i in range(len(soup.find_all('div', attrs={'class': 'customer-review__review-item__copy'}))):
                brand.append('Sealy')
                model.append(soup.find('h2', attrs={'class': 'configurator__title'}).text)
                rating.append(soup.find_all('span', attrs={'class': 'ada-screenreader-only'})[i+9].text)
                review.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__copy'})[i].text)
                warranty.append('10-Year')
                trial.append(100)
                reviewer_name.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'})[i].text.split(',')[0])
                review_location.append(','.join(soup.find_all('div', attrs={'class': 'customer-review__review-item__review-author'})[i].text.split(',')[1:]))
                review_date.append(soup.find_all('div', attrs={'class': 'customer-review__review-item__date'})[0].text)
                count += 1

                if count % 100 == 0:
                    print('{}/{} reviews added'.format(count, total_reviews))
            
            
    return brand, model, rating, review, warranty, trial, reviewer_name, review_location, review_date


In [121]:
df = pd.DataFrame()

df['brand'] = brand
df['model'] = model
df['rating'] = rating
df['review'] = review
df['warranty'] = warranty
df['trial'] = trial
df['reviewer_name'] = reviewer_name
df['review_location'] = review_location
df['review_date'] = review_date

In [124]:
df.to_csv('Sealy_Reviews.csv')

In [138]:
def get_reviews_2(company, number_of_reviews):
    brand = []
    price = []
    rating = []
    review = []
    queen_size = []
    warranty = []
    trial = []
    reviewer_name = []
    review_location = []
    review_date = []


    url = 'https://purple.com/mattresses/purple-bed'
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(5)

    # Click dropdown menu
    element = browser.find_element_by_xpath('//*[@id="stamped-sort-select"]')
    browser.execute_script("arguments[0].click();", element)

    # Click most recent on drop down menu
    element = browser.find_element_by_xpath('//*[@id="stamped-sort-select"]/option[3]')
    browser.execute_script("arguments[0].click();", element)

    total_reviews = 500
    count = 0

    while count < total_reviews:
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        count+= len(soup.find_all(class_='stamped-starratings stamped-review-header-starratings'))
        for i in range(len(soup.find_all(class_='stamped-starratings stamped-review-header-starratings'))):
            brand.append('purple')
            price.append(1049)
            c = collections.Counter(soup.find_all(class_='stamped-starratings stamped-review-header-starratings')[i])
            c_2 = list(c)
            c_key = c_2[1]
            rating.append(collections.Counter(soup.find_all(class_='stamped-starratings stamped-review-header-starratings')[i])[c_key])
            review.append(soup.find_all(class_='stamped-review-content-body')[i].text)
            queen_size.append('')
            warranty.append('10-Year')
            trial.append(100)
            reviewer_name.append(soup.find_all('div', attrs={'class': 'author'})[i])
            review_location.append(soup.find_all('div', attrs={'class': 'review-location'})[i].text)
            review_date.append(soup.find_all('div', attrs={'class': 'created'})[i].text)

        element = browser.find_element_by_xpath('//*[@id="stamped-main-widget"]/div/div[3]/div[5]/ul/li[14]/a')
        browser.execute_script("arguments[0].click();", element)

    return brand, price, rating, review, queen_size, warranty, trial, reviewer_name, review_location, review_date

AC Pacific: Reviews and info on Amazon

Sealy: Sealy website

In [128]:
import glob
files = glob.glob('*.csv')

df_list = []

for file in files:
    data = pd.read_csv(file, lineterminator='\n')
    df_list.append(data)
    
df = pd.concat(df_list)

In [129]:
df

,Unnamed: 0,brand,model,rating,review,warranty,trial,reviewer_name,review_location,review_date
0,0,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,10-Year,100,"<div class=""author"">Jessica G.</div>",United States,02/22/2021
1,1,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,10-Year,100,"<div class=""author"">Jared B.</div>","Warrenton, Virginia",09/19/2017
2,2,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,10-Year,100,"<div class=""author"">LeAnn M.</div>",United States,03/05/2021
3,3,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,10-Year,100,"<div class=""author"">Kathryn W.</div>",United States,03/04/2021
4,4,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,10-Year,100,"<div class=""author"">Adam R.</div>",United States,03/04/2021
...,...,...,...,...,...,...,...,...,...,...
5093,5093,Casper,Element Mattress,5.0,My granddaughter finds her new twin very comfo...,10-year,100,Tricia,"Clermont, FL","Jul 24, 2018"
5094,5094,Casper,Element Mattress,5.0,"After years and years of sleeping on coils, my...",10-year,100,Jared,"Los Angeles, CA","Jul 24, 2018"
5095,5095,Casper,Element Mattress,5.0,I have had a hard time sleeping since I broke ...,10-year,100,Megan,"LARAMIE, WY","Jul 24, 2018"
5096,5096,Casper,Element Mattress,4.0,Firmer than I expected but best sleep ever. I ...,10-year,100,Joshua,"Robinson, IL","Jul 24, 2018"
